**Predict the effect of Genetic Variants to enable Personalized Medicine**

In [1]:
import os
import tqdm
import string
import pandas as pd
import numpy as np
import nltk
import re
from random import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.utils import np_utils
from keras.preprocessing import text, sequence
from keras.callbacks import TensorBoard
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

Using TensorFlow backend.


In [2]:
np.random.seed(42)

**Reading Data**


In [3]:
# Read Data
train_variant = pd.read_csv("training_variants")
test_variant = pd.read_csv("test_variants")
train_text = pd.read_csv("training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text = pd.read_csv("test_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

# Join text and variant data on ID
train = pd.merge(train_variant, train_text, how='left', on='ID')

# Drop Class labels from training data
train_y = train['Class'].values
train_x = train.drop('Class', axis=1)

#  Check shape of training data
train_size = train_x.shape[0]
# number of train data : 3321

# Join text and variant data on ID
test_x = pd.merge(test_variant, test_text, how='left', on='ID')

# Check shape and size of test data
test_size = test_x.shape[0]
test_index = test_x['ID'].values
# number of test data : 5668

# Join all data for analysis
all_data = np.concatenate((train_x, test_x), axis=0)
all_data = pd.DataFrame(all_data)
all_data.columns = ["ID", "Gene", "Variation", "Text"]

In [4]:
test_text.head()

,ID,Text
0,0,2. This mutation resulted in a myeloproliferat...
1,1,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,Vascular endothelial growth factor receptor (V...
3,3,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,Abstract Retinoblastoma is a pediatric retina...


In [5]:
all_data.head()

,ID,Gene,Variation,Text
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...


**Text Cleaning**

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/olliegraham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [8]:
common_stop = ["fig", "figure", "et", "al", "al.", "also",
                "data", "analyze", "study", "table", "using",
                "method", "result", "conclusion", "author", 
                "find", "found", "show"]

In [5]:
# Define text pre-processing funtions

# Constructs a labelled list of sentences from a text corpus
def constructLabeledSentences(data):
    sentences=[]
    for index, row in data.iteritems():
        sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
    return sentences

# Converts text to lower case and removes punctuation
def cleanup(text):
    text = text.lower()
    text = text.translate(str.maketrans("","", string.punctuation))
    return text

# Removes punctuations, converts text to lower case, splits words into tokens, removes stopwords and digits from text
def textClean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and not w.isdigit()]    
    text = " ".join(text)
    return(text)


In [10]:
%%time

all_data['Txt'] = all_data.Text.apply(cleanup)

CPU times: user 54.5 s, sys: 4.45 s, total: 59 s
Wall time: 1min 11s


In [6]:
%%time

all_data['TxtNS'] = all_data.Text.apply(textClean)

CPU times: user 45 s, sys: 1.24 s, total: 46.2 s
Wall time: 46.7 s


In [7]:
allTextNS = all_data['TxtNS'].apply(cleanup)

In [11]:
allText = all_data['Txt']

In [14]:
allTextNS = all_data['TxtNS']

In [15]:
sentences = constructLabeledSentences(allText)

In [9]:
sentences2 = constructLabeledSentences(allTextNS)

In [8]:
print(allTextNS[0])

cyclindependent kinases cdks regulate variety fundamental cellular processes cdk10 stands one last orphan cdks activating cyclin identified kinase activity revealed previous work shown cdk10 silencing increases ets2 vets erythroblastosis virus e26 oncogene homolog driven activation mapk pathway confers tamoxifen resistance breast cancer cells precise mechanisms cdk10 modulates ets2 activity generally functions cdk10 remain elusive demonstrate cdk10 cyclindependent kinase identifying cyclin activating cyclin cyclin m orphan cyclin product fam58a whose mutations cause star syndrome human developmental anomaly whose features include toe syndactyly telecanthus anogenital renal malformations show star syndromeassociated cyclin mutants unable interact cdk10 cyclin silencing phenocopies cdk10 silencing increasing craf conferring tamoxifen resistance breast cancer cells cdk10cyclin phosphorylates ets2 vitro cells positively controls ets2 degradation proteasome ets2 protein levels increased cel

In [60]:
print(sentences2[0])

LabeledSentence(['cyclindependent', 'kinases', 'cdks', 'regulate', 'variety', 'fundamental', 'cellular', 'processes', 'cdk10', 'stands', 'one', 'last', 'orphan', 'cdks', 'activating', 'cyclin', 'identified', 'kinase', 'activity', 'revealed', 'previous', 'work', 'shown', 'cdk10', 'silencing', 'increases', 'ets2', 'vets', 'erythroblastosis', 'virus', 'e26', 'oncogene', 'homolog', 'driven', 'activation', 'mapk', 'pathway', 'confers', 'tamoxifen', 'resistance', 'breast', 'cancer', 'cells', 'precise', 'mechanisms', 'cdk10', 'modulates', 'ets2', 'activity', 'generally', 'functions', 'cdk10', 'remain', 'elusive', 'demonstrate', 'cdk10', 'cyclindependent', 'kinase', 'identifying', 'cyclin', 'activating', 'cyclin', 'cyclin', 'm', 'orphan', 'cyclin', 'product', 'fam58a', 'whose', 'mutations', 'cause', 'star', 'syndrome', 'human', 'developmental', 'anomaly', 'whose', 'features', 'include', 'toe', 'syndactyly', 'telecanthus', 'anogenital', 'renal', 'malformations', 'show', 'star', 'syndromeassocia

**Featurizing using Doc2Vec**

In [10]:
INPUT_DIM=300

In [11]:
# Create text features using Doc2Vec

doc2vec_epocs=10 #TODO Change to 100
model=None
filename='vectors/docEmbeddings_nostop_all_10.d2v'
if os.path.isfile(filename):
    model = Doc2Vec.load(filename)
else:
    model = Doc2Vec(min_count=1, window=5, size=INPUT_DIM, sample=1e-4, negative=5, workers=8, iter=doc2vec_epocs, seed=1)
    model.build_vocab(sentences)
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    model.save(filename)

In [12]:
model.most_similar('mutation')

[('mutations', 0.6853571534156799),
 ('substitution', 0.6260187029838562),
 ('variant', 0.5576804280281067),
 ('mutant', 0.48260611295700073),
 ('alteration', 0.4624151885509491),
 ('transversion', 0.4485575556755066),
 ('polymorphism', 0.41885772347450256),
 ('allele', 0.4183550477027893),
 ('patient', 0.4144909381866455),
 ('mutational', 0.4105629324913025)]

**Data Preparation**

In [13]:
train_arrays = np.zeros((train_size, INPUT_DIM))
test_arrays = np.zeros((test_size, INPUT_DIM))

for i in range(train_size):
    train_arrays[i] = model.docvecs['Text_'+ str(i)]

label_encoder = LabelEncoder()
label_encoder.fit(train_y)
encoded_y = np_utils.to_categorical((label_encoder.transform(train_y)))

j=0
for i in range(train_size, train_size + test_size):
    test_arrays[j] = model.docvecs['Text_'+ str(i)]
    j=j+1

In [38]:
print(encoded_y[0])

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.]


** Gene and Variation Featurizer **

In [ ]:
Gene_INPUT_DIM=25

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=25, n_iter=Gene_INPUT_DIM, random_state=12)

one_hot_gene = pd.get_dummies(all_data['Gene'])
truncated_one_hot_gene = svd.fit_transform(one_hot_gene.values)

one_hot_variation = pd.get_dummies(all_data['Variation'])
truncated_one_hot_variation = svd.fit_transform(one_hot_variation.values)

** Merge Input features **

In [ ]:
train_set=np.hstack((truncated_one_hot_gene[:train_size],truncated_one_hot_variation[:train_size], train_arrays))
test_set=np.hstack((truncated_one_hot_gene[train_size:],truncated_one_hot_variation[train_size:], test_arrays))
print(train_set[0][:50])

**Training Model**

In [14]:
# define initial model
def initial_model():
    model = Sequential()
    model.add(Dense(512, input_dim=INPUT_DIM, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(9, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
model = initial_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               154112    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 80)                41040     
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 729       
Total params: 195,881
Trainable params: 195,881
Non-trainable params: 0
_________________________________________________________________


In [30]:
tb = TensorBoard(log_dir='./logs/run2', histogram_freq=1, batch_size=64, write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [31]:
estimator = KerasClassifier(build_fn=initial_model, epochs=10, batch_size=64)
estimator.fit(train_arrays, encoded_y, validation_split=0.05, callbacks=[tb])

Train on 3154 samples, validate on 167 samples
Epoch 1/10
3154/3154 [==============================] - 1s - loss: 1.4847 - acc: 0.4686 - val_loss: 1.5648 - val_acc: 0.3593
Epoch 2/10
3154/3154 [==============================] - 0s - loss: 1.0429 - acc: 0.6338 - val_loss: 1.5472 - val_acc: 0.4132
Epoch 3/10
3154/3154 [==============================] - 0s - loss: 0.8812 - acc: 0.6864 - val_loss: 1.5969 - val_acc: 0.4731
Epoch 4/10
3154/3154 [==============================] - 0s - loss: 0.7472 - acc: 0.7353 - val_loss: 1.7586 - val_acc: 0.4611
Epoch 5/10
3154/3154 [==============================] - 0s - loss: 0.6562 - acc: 0.7609 - val_loss: 1.8426 - val_acc: 0.4012
Epoch 6/10
3154/3154 [==============================] - 0s - loss: 0.5800 - acc: 0.7955 - val_loss: 1.9560 - val_acc: 0.4491
Epoch 7/10
3154/3154 [==============================] - 0s - loss: 0.5389 - acc: 0.7974 - val_loss: 1.9681 - val_acc: 0.4731
Epoch 8/10
3154/3154 [==============================] - 0s - loss: 0.4763 - ac

In [33]:
y_pred = estimator.predict_proba(test_arrays)

5376/5668 [===========================>..] - ETA: 0s

In [34]:
estimator.score(test_arrays, y_pred)

5440/5668 [===========================>..] - ETA: 0s

0.18260409317558385

** Advanced Model **

In [68]:
# define model
def baseline_model():
    model = Sequential()
    model.add(Dense(512, input_dim=INPUT_DIM, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(9, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [69]:
model2 = baseline_model()
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 512)               154112    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 512)               262656    
__________

In [70]:
estimator2 = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=64)
estimator2.fit(train_arrays, encoded_y, validation_split=0.05)

Train on 3154 samples, validate on 167 samples
Epoch 1/10
3154/3154 [==============================] - 3s - loss: 1.5674 - acc: 0.4230 - val_loss: 1.6184 - val_acc: 0.4551
Epoch 2/10
3154/3154 [==============================] - 1s - loss: 1.1729 - acc: 0.5780 - val_loss: 1.7050 - val_acc: 0.4970
Epoch 3/10
3154/3154 [==============================] - 1s - loss: 1.0056 - acc: 0.6297 - val_loss: 1.7387 - val_acc: 0.3174
Epoch 4/10
3154/3154 [==============================] - 1s - loss: 0.8447 - acc: 0.6937 - val_loss: 1.8244 - val_acc: 0.3413
Epoch 5/10
3154/3154 [==============================] - 1s - loss: 0.7305 - acc: 0.7346 - val_loss: 2.3056 - val_acc: 0.4072
Epoch 6/10
3154/3154 [==============================] - 1s - loss: 0.6343 - acc: 0.7619 - val_loss: 2.2701 - val_acc: 0.3533
Epoch 7/10
3154/3154 [==============================] - 1s - loss: 0.6022 - acc: 0.7828 - val_loss: 2.0332 - val_acc: 0.4551
Epoch 8/10
3154/3154 [==============================] - 1s - loss: 0.5436 - ac

In [71]:
y_pred = estimator2.predict_proba(test_arrays)

5632/5668 [============================>.] - ETA: 0s

In [72]:
estimator2.score(test_arrays, y_pred)

5376/5668 [===========================>..] - ETA: 0s

0.19936485535970605

**Create Submission File**

In [73]:
""" Submission """
submission = pd.DataFrame(y_pred)
submission['id'] = test_index
submission.columns = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9', 'id']
submission.to_csv("submissions/initial_nostop.csv",index=False)

In [ ]:
estimator.score(test_set, y_pred)

** Naive Predictor **

In [ ]:
percent = train['Class'].value_counts() / float(len(train)) * 100

tab = pd.DataFrame({
    "Class": percent.index,
    "Percent of Total": round(percent, 2)})

tab.sort_values(by="Class")
tab

In [ ]:
y_naive = []

for i in range(len(test_set)):
    y_naive.append(percent/100)
    
y_naive[1]

In [ ]:
""" Naive Submission """
submission = pd.DataFrame(y_naive)
submission['id'] = test_index
submission.columns = ['class7', 'class4', 'class1', 'class2', 'class6', 'class5', 'class3', 'class9', 'class8', 'id']
submission.to_csv("naive_submission.csv",index=False)